## 1. 模块库的引入

In [1]:
from langchain_community.document_loaders import PyPDFLoader  # PDF文档提取
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 文档拆分chunk
import faiss  # faiss向量库
import numpy as np  # 处理嵌入向量数据，用于faiss向量检索
import os  # 引入操作系统库，后续配置环境变量和获取文件路径
from dotenv import load_dotenv

load_dotenv()
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 不使用分词并行化操作，避免多线程或多进程环境中运行多个模型引发冲突

# 设置Qwen系列具体模型及对应的调用API密钥，从硅基流动大模型服务平台获得
chat_model = os.getenv('chat_model')
embed_model = os.getenv('embed_model')
api_key = os.getenv('api_key')
base_url = os.getenv('base_url')

Python-dotenv could not parse statement starting at line 1


## 2. 索引流程

In [2]:
from openai import OpenAI

### 2.1 获取文本嵌入

In [3]:
def load_embedding_model():
    """
    加载bge-m3模型
    :return: 返回bge-m3模型嵌入的结果
    """
    client = OpenAI(
            api_key=api_key, # 从https://cloud.siliconflow.cn/account/ak获取
            base_url=base_url
        )
    print(f"加载Embedding模型中")

    def get_embedding(text):
       text = text.replace("\n", " ")
       return client.embeddings.create(input = [text], model=embed_model).data[0].embedding

    return get_embedding

### 2.2 建立索引，即把文本嵌入写入FAISS向量数据库

In [4]:
def indexing_process(pdf_file, embedding_model):
    """
    索引流程：加载PDF文件，并将其内容分割成小块，计算这些小块的嵌入向量并将其存储在FAISS向量数据库中。
    :param pdf_file: PDF文件路径
    :param embedding_model: 预加载的嵌入模型
    :return: 返回Faiss嵌入向量索引和分割后的文本块原始内容列表
    """
    # PyPDFLoader加载PDF文件，忽略图片提取
    pdf_loader = PyPDFLoader(pdf_file, extract_images=False)
    # 配置RecursiveCharacterTextSplitter分割文本块库参数，每个文本块的大小为512字符（非token），相邻文本块之间的重叠128字符（非token）
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512, chunk_overlap=128
    )
    # 加载PDF文档,提取所有页的文本内容
    pdf_content_list = pdf_loader.load()
    # 将每页的文本内容用换行符连接，合并为PDF文档的完整文本
    pdf_text = "\n".join([page.page_content for page in pdf_content_list])
    print(f"PDF文档的总字符数: {len(pdf_text)}")

    # 将PDF文档文本分割成文本块Chunk
    chunks = text_splitter.split_text(pdf_text)
    print(f"分割的文本Chunk数量: {len(chunks)}")

    # 文本块转化为嵌入向量列表，normalize_embeddings表示对嵌入向量进行归一化，用于准确计算相似度
    embeddings = []
    for chunk in chunks:
        embedding = embedding_model(chunk)
        embeddings.append(embedding)

    print("文本块Chunk转化为嵌入向量完成")

    # 将嵌入向量列表转化为numpy数组，Faiss向量库操作需要numpy数组输入
    embeddings_np = np.array(embeddings)

    # 获取嵌入向量的维度（每个向量的长度）
    dimension = embeddings_np.shape[1]

    # 使用余弦相似度创建FAISS索引
    index = faiss.IndexFlatIP(dimension)
    # 将所有的嵌入向量添加到FAISS索引中，后续可以用来进行相似性检索
    index.add(embeddings_np)

    print("索引过程完成.")

    return index, chunks

## 3. 检索流程

In [5]:
def retrieval_process(query, index, chunks, embedding_model, top_k=3):
    """
    检索流程：将用户查询Query转化为嵌入向量，并在Faiss索引中检索最相似的前k个文本块。
    :param query: 用户查询语句
    :param index: 已建立的Faiss向量索引
    :param chunks: 原始文本块内容列表
    :param embedding_model: 预加载的嵌入模型
    :param top_k: 返回最相似的前K个结果
    :return: 返回最相似的文本块及其相似度得分
    """
    # 将查询转化为嵌入向量
    query_embedding = embedding_model(query)
    # 将嵌入向量转化为numpy数组，Faiss索引操作需要numpy数组输入
    query_embedding = np.array([query_embedding])

    # 在 Faiss 索引中使用 query_embedding 进行搜索，检索出最相似的前 top_k 个结果。
    # 返回查询向量与每个返回结果之间的相似度得分（在使用余弦相似度时，值越大越相似）排名列表distances，最相似的 top_k 个文本块在原始 chunks 列表中的索引indices。
    distances, indices = index.search(query_embedding, top_k)

    print(f"查询语句: {query}")
    print(f"最相似的前{top_k}个文本块:")

    # 输出查询出的top_k个文本块及其相似度得分
    results = []
    for i in range(top_k):
        # 获取相似文本块的原始内容
        result_chunk = chunks[indices[0][i]]
        print(f"文本块 {i}:\n{result_chunk}")

        # 获取相似文本块的相似度得分
        result_distance = distances[0][i]
        print(f"相似度得分: {result_distance}\n")

        # 将相似文本块存储在结果列表中
        results.append(result_chunk)

    print("检索过程完成.")
    return results

## 4. 生成流程

In [6]:
def generate_process(query, chunks):
    """
    生成流程：调用Qwen大模型云端API，根据查询和文本块生成最终回复。
    :param query: 用户查询语句
    :param chunks: 从检索过程中获得的相关文本块上下文chunks
    :return: 返回生成的响应内容
    """
    # 构建参考文档内容，格式为“参考文档1: \n 参考文档2: \n ...”等
    context = ""
    for i, chunk in enumerate(chunks):
        context += f"参考文档{i+1}: \n{chunk}\n\n"

    # 构建生成模型所需的Prompt，包含用户查询和检索到的上下文
    prompt = f"根据参考文档回答问题：{query}\n\n{context}"
    print(f"生成模型的Prompt: {prompt}")

    # 准备请求消息，将prompt作为输入
    messages = [{'role': 'user', 'content': prompt}]

    # 调用大模型API实例化大语言模型
    client = OpenAI(
        api_key=api_key, # 从https://cloud.siliconflow.cn/account/ak获取
        base_url=base_url
    )

    try:
        # 使用大模型生成响应
        responses = client.chat.completions.create(
        model=chat_model,
        messages = messages,
        temperature=0.01,
        top_p=0.95,
        stream=False,
        )

        print("生成过程开始:")
        # 流式响应
        # 初始化变量以存储生成的响应内容
        # generated_response = ""

        # # 逐步获取和处理模型的增量输出
        # for response in responses:
        #     response = response.choices[0].delta.content
        #     if response is not None:
        #         generated_response+=response
        #         print(response, end="")

        # 非流式响应
        generated_response = responses.choices[0].message.content
        print(generated_response)

        print("\n生成过程完成.")
        return generated_response
    except Exception as e:
        print(f"大模型生成过程中发生错误: {e}")
        return None

## 5. 执行整个流程

In [7]:
print("RAG过程开始.")
query="下面报告中涉及了哪几个行业的案例以及总结各自面临的挑战？"

RAG过程开始.


### 1. 执行索引过程

In [8]:
embedding_model = load_embedding_model()

# 索引流程：加载PDF文件，分割文本块，计算嵌入向量，存储在FAISS向量库中（内存）
index, chunks = indexing_process('data/lesson1.pdf', embedding_model)

加载Embedding模型中
PDF文档的总字符数: 9163
分割的文本Chunk数量: 24
文本块Chunk转化为嵌入向量完成
索引过程完成.


### 2. 执行检索过程

In [9]:
# 检索流程：将用户查询转化为嵌入向量，检索最相似的文本块
retrieval_chunks = retrieval_process(query, index, chunks, embedding_model)

查询语句: 下面报告中涉及了哪几个行业的案例以及总结各自面临的挑战？
最相似的前3个文本块:
文本块 0:
面的数字化转型。
2.3.2 面临的挑战
在数字化转型之前，金融业案例中银行面临以下主要挑战：客户服务模式过时，主要依赖实
体网点，导致服务效率低、客户体验差；金融科技企业带来巨大竞争压力，凭借创新技术和
便捷服务吸引大量客户，尤其是年轻一代；数据孤岛和风险管理滞后，各业务部门缺乏数据
共享机制，导致信息无法整合，风险管理效率低。
2.3.3 数字化转型解决方案
为应对金融业案例挑战，银行实施了多方面的数字化转型措施：首先，构建数字化银行平台，
推出移动银行应用、在线服务、虚拟客服和智能理财顾问，显著提升了服务便捷性和客户满
意度；其次，引入人工智能和大数据分析技术，通过个性化金融产品推荐和实时风险监控，
提升客户服务质量和风险管理能力。
相似度得分: 0.5160303115844727

文本块 1:
发展和应用，企业在环境保护和可持续发展领域的能力将进一步增强，为全球的可持续发展
目标做出更大的贡献。这不仅有助于企业在激烈的市场竞争中保持领先地位，还将推动全社
会向着更加绿色、更加可持续的未来迈进。
1.3 总结
总的来说，数字化转型不仅是企业应对当前市场挑战的一种战略选择，更是通向未来的必由
之路。成功的数字化转型将帮助企业提升核心竞争力，优化客户体验，并在全球经济中获得
长期的可持续发展。
2. 案例分析
2.1 案例一：制造业的数字化转型
2.1.1 公司背景
制造业案例介绍了一家成立于 20 世纪初的德国老牌汽车制造公司，拥有悠久的历史和丰富
的制造经验。面对日益激烈的市场竞争和消费者需求的变化，公司意识到传统制造模式已无
法适应现代市场需求，因而决定实施全面的数字化转型，以保持竞争力。
2.1.2 面临的挑战
在数字化转型之前，制造业案例公司面临多重挑战：生产效率低下，传统制造流程依赖人工，
导致效率低且易出错；供应链复杂，涉及多个国家和地区，信息传递不及时，造成库存管理
困难，甚至存在供应链断裂的风险；客户需求变化快，传统大规模生产方式无法满足市场对
个性化定制产品的需求。
相似度得分: 0.5159369707107544

文本块 2:
统一、客户体验不一致，难以提供无缝购物体验；数据利用率低，尽管拥有大量消费者和销
售数据，但缺乏先进的

### 3. 生成回答

In [10]:
# 生成流程：调用Qwen大模型生成响应
generate_process(query, retrieval_chunks)

print("RAG过程结束.")

生成模型的Prompt: 根据参考文档回答问题：下面报告中涉及了哪几个行业的案例以及总结各自面临的挑战？

参考文档1: 
面的数字化转型。
2.3.2 面临的挑战
在数字化转型之前，金融业案例中银行面临以下主要挑战：客户服务模式过时，主要依赖实
体网点，导致服务效率低、客户体验差；金融科技企业带来巨大竞争压力，凭借创新技术和
便捷服务吸引大量客户，尤其是年轻一代；数据孤岛和风险管理滞后，各业务部门缺乏数据
共享机制，导致信息无法整合，风险管理效率低。
2.3.3 数字化转型解决方案
为应对金融业案例挑战，银行实施了多方面的数字化转型措施：首先，构建数字化银行平台，
推出移动银行应用、在线服务、虚拟客服和智能理财顾问，显著提升了服务便捷性和客户满
意度；其次，引入人工智能和大数据分析技术，通过个性化金融产品推荐和实时风险监控，
提升客户服务质量和风险管理能力。

参考文档2: 
发展和应用，企业在环境保护和可持续发展领域的能力将进一步增强，为全球的可持续发展
目标做出更大的贡献。这不仅有助于企业在激烈的市场竞争中保持领先地位，还将推动全社
会向着更加绿色、更加可持续的未来迈进。
1.3 总结
总的来说，数字化转型不仅是企业应对当前市场挑战的一种战略选择，更是通向未来的必由
之路。成功的数字化转型将帮助企业提升核心竞争力，优化客户体验，并在全球经济中获得
长期的可持续发展。
2. 案例分析
2.1 案例一：制造业的数字化转型
2.1.1 公司背景
制造业案例介绍了一家成立于 20 世纪初的德国老牌汽车制造公司，拥有悠久的历史和丰富
的制造经验。面对日益激烈的市场竞争和消费者需求的变化，公司意识到传统制造模式已无
法适应现代市场需求，因而决定实施全面的数字化转型，以保持竞争力。
2.1.2 面临的挑战
在数字化转型之前，制造业案例公司面临多重挑战：生产效率低下，传统制造流程依赖人工，
导致效率低且易出错；供应链复杂，涉及多个国家和地区，信息传递不及时，造成库存管理
困难，甚至存在供应链断裂的风险；客户需求变化快，传统大规模生产方式无法满足市场对
个性化定制产品的需求。

参考文档3: 
统一、客户体验不一致，难以提供无缝购物体验；数据利用率低，尽管拥有大量消费者和销
售数据，但缺乏先进的数据分析工具，未能转化为可操作的商业洞察。
2.2.3 数字化转型解决方案
为了解决